<h3 align = center>How to make a simple video using FFMPEG</h3>
last update: Sept 2025<br>

- [Package used](#Package-used)
- [Trimming a video](#Trimming-a-video)
- [Splitting a video into section](#Splitting-a-video-into-sections)
- [Creating a black still](#Creating-a-black-still)
- [concatenating videos](#concatenating-videos)
- [Reencoding](#Reencoding)
- [Separating different audio channels from a video](#Separating-different-audio-channels-from-a-video)
- [extracting voice to subtitles](#extracting-voice-to-subtitles)
- [Translation](#Translation)
- [combining translations for proofreading](#combining-translations-for-proofreading)
- [Separating translation after proofreading](#Separating-translation-after-proofreading)
- [creating zoom video](#creating-zoom-video)
- [Overlaying everything](#Overlaying-everything)
- [extracting screen shot](#extracting-screen-shot)
- [srt to ass](#srt-to-ass)
- [adding mosaic](#adding-mosaic)
- [burning in subtitle](#Burning-in-subtitle)
- [creating end credis](#Creating-End-Credits)
- [final concatenation](#final-concatenation)
- [Video Codec](#Printing-video-info)
- [parse time](#parse-time)
- [media dimensions](#media-dimensions)
- [mp3 to wav](#mp3-to-wav)
- [adjust wav volume](#adjust-wav-volume)
- [extract pages to txt](#extract-pages-to-txt)
- [pip3 download](#pip3-download)


## Package used:
<i>ffmpeg, ffprobe,re, datetime, math, json, PIL, fractions, matplotlib, tempfile, pdfplumber,zipfile <br>
shutil, numpy, openai-whisper, deep_translator, deepl, openai, demucs, opencc, opencv-python</i> <br>

whisper environment: 
 <i>openai-whisper</i> , <i>demucs</i>, <i>deepl</i>, <i>openai</i> <br>

How to install:<br>
<i>!pip3 install openai-whisper</i>

The code we are using here is in <b><i>video_commands.py</i></b>.


### Trimming a video
we want to extract <i>11:00 to 16:00</i> :<br>
<i>ffmpeg -y -ss 11:00 -to 16:00 -i Marines_at_Tarawa_Return_to_Guam.mp4 -c:v libx264 -preset veryfast -crf 18 -c:a copy  marines_5min.mov</i> <br><br>




In [ ]:
from video_commands import * 
main_video = "Marines_at_Tarawa_Return_to_Guam.mp4"
start_time = "11:00" #you can enter as 1)an interger which is second, 2)mm:ss, or 3)hh:mm:ss
end_time = "16:00" 
output_file = "marines_5min.mov"
trim_video(main_video, start_time, end_time, output_file,crf=23, preset = "fast")

In [ ]:
from video_commands import * 
main_video = ""
start_time = "11:00" #you can enter as 1)an interger which is second, 2)mm:ss, or 3)hh:mm:ss
end_time = "16:00" 
output_file = ""
trim_video(main_video, start_time, end_time, output_file,crf=23, preset = "fast")

## Printing video info
ffprobe code for video: <br>
<i>ffprobe -v error -select_streams v:0 -show_entries stream=codec_name,width,height,r_frame_rate,pix_fmt -of csv=p=0 marines_5min.mov</i><br>
ffmpeg code for audio: <br>
<i>ffprobe -v error -select_streams a:0 -show_entries stream=codec_name,sample_rate,channels -of csv=p=0 marines_5min.mov</i>
<br><br>
Now let's run the python code below:<br>

In [ ]:
file_list = ["marines_5min.mov"]
print_media_info(file_list)

In [ ]:
from video_commands import * 
file_list = [""]
print_media_info(file_list)

## Splitting a video into sections
4 sections:<br>
1) 0:00 - 1:30 <br>
2) 1:30 - 2:30 <br>
3) 2:30 - 4:00 <br>
4) 4:00 - the end <br>

<i>ffmpeg -y -ss 0.000 -i marines_5min.mov -t 90.000 -c:v libx264 -preset fast -c:a pcm_s16le -ar 48000 -ac 2 -movflags +faststart video1.mov</i><br>
<i> ffmpeg -y -ss 90.000 -i marines_5min.mov -t 60.000 -c:v libx264 -preset fast -c:a pcm_s16le -ar 48000 -ac 2 -movflags +faststart video2.mov</i><br>
<i>ffmpeg -y -ss 150.000 -i marines_5min.mov -t 90.000 -c:v libx264 -preset fast -c:a pcm_s16le -ar 48000 -ac 2 -movflags +faststart video3.mov </i><br>
<i>ffmpeg -y -ss 240.000 -i marines_5min.mov -t 60.009 -c:v libx264 -preset fast -c:a pcm_s16le -ar 48000 -ac 2 -movflags +faststart video4.mov
 </i><br><br>

 Below is the python code to implement the above. Let's run it.<br>


In [ ]:
main_video = "marines_5min.mov"
the_end = get_video_length(main_video) #we retrive the exact end time in hh:mm:ss by calling this function
sections = ["0:00 - 1:30","1:30 - 2:30","2:30 - 4:00",f"4:00-{the_end}"]
#Below are the output files. The length of this list has to match the length of the above list "sections"
output_files = ["video1.mov","video2.mov","video3.mov","video4.mov"] 
split_video(main_video, sections, output_files, audio_codec="wav")

In [ ]:
from video_commands import * 
main_video = ".mov"
the_end = get_video_length(main_video) #we retrive the exact end time in hh:mm:ss by calling this function
sections = ["0:00 - 1:30","1:30 - 2:30","2:30 - 4:00",f"4:00-{the_end}"]
#Below are the output files. The length of this list has to match the length of the above list "sections"
output_files = ["video1.mov","video2.mov","video3.mov","video4.mov"] 
split_video(main_video, sections, output_files, audio_codec="wav")

## Creating a black still
creating blank black still:<br>

<i>ffmpeg -y -f lavfi -i color=c=black:s=556x412:r=25.0:d=5 -f lavfi -i anullsrc=r=44100:cl=stereo -shortest -c:v libx264 -pix_fmt yuv420p -c:a pcm_s16le temp_black.mp4</i><br>

Then python creates an ass file (a subtitle format, this is a text file) with the text called <i>temp_sub.ass</i>. Finally <i>temp_sub.ass</i> is burnt in to <i>temp_black.mp4</i> and the final output is what we want.

<i>ffmpeg -y -i temp_black.mp4 -vf ass=temp_sub.ass -c:v libx264 -pix_fmt yuv420p -c:a copy black1.mov</i>

<b>Note that if the numbers(like <i>1</i> in <i>"Section 1"</i>) appear strange (maybe bigger than the text) in <i>black1.mov</i>, you need to install some fonts. I don't remember how to do it, but I do remember I asked chatgpt.<br></b>


In [ ]:
main_video = "marines_5min.mov"
text = ["Section 1","Section 2","Section 3"]
output_files = ["black1.mov","black2.mov","black3.mov"]
duration = 5 # in seconds
for txt,output_names in zip(text,output_files):
    output_file = create_black_still(main_video,txt,duration,output_names,font_name="Arial",font_size=72,
    font_color="&H00FFFFFF" #Solid White
)

In [ ]:
from video_commands import * 
main_video = ".mov"
text = ["","",""]
output_files = ["black1.mov","black2.mov","black3.mov"]
duration = 5 # in seconds
for txt,output_names in zip(text,output_files):
    output_file = create_black_still(main_video,txt,duration,output_names,font_name="Arial",font_size=72,
    font_color="&H00FFFFFF" #Solid White

## Reencoding

Here is the break down of <i>reencode_to_match()</i>:<br>
1)Get main video's codec: <br>
<i>ffprobe -v error -select_streams v:0 -show_entries stream=width,height,r_frame_rate,duration -of json marines_5min.mov
</i><br>

The codec is retrieved and put into the next step.<br>
            
2)Re-encode the main video to audio = PCM: <br>
<i>ffmpeg -y -i media_files/marines_5min.mov -r 25.0 -c:v libx264 -c:a pcm_s16le -ar 44100 -ac 2 -preset fast -crf 18 marines_5min_tmp.mov</i><br><br>
3)Rescale video to match the main video's dimension (yes, the main video is rescaled to the main video, it does the same to all other videos):<br>
<i>ffmpeg -y -i marines_5min_tmp.mov -vf scale=556:412,pad=556:412:0:0:black -c:v libx264 -preset fast -crf 18 -c:a copy marines_5min_reencoded.mov</i><br><br>
4)Pad/truncate audio to match video duration:<br>
<i>ffmpeg -y -i marines_5min_reencoded.mov -c:v copy -af apad,atrim=0:300.000000 marines_5min_reencoded_padded.mov</i><br>

So the final product we want is the last one - <i>marines_5min_reencoded_padded.mov</i>. The same procedure applies to other videos.



In [ ]:
main_video = "marines_5min.mov"
# The main video serves as the "standard" codec for others to follow
list_to_reencode = [main_video,"video1.mov","video2.mov","video3.mov","video4.mov",
            "black1.mov","black2.mov","black3.mov"]
reencoded_file_names, reencoded_file_dict =reencode_to_match(main_video, list_to_reencode,crf="23", preset="fast")


In [ ]:
from video_commands import * 
main_video = ".mov"
# The main video serves as the "standard" codec for others to follow
list_to_reencode = [main_video,".mov",".mov",".mov",".mov"]
reencoded_file_names, reencoded_file_dict =reencode_to_match(main_video, list_to_reencode,crf="23", preset="fast")

### concatenating videos

In [ ]:
#input the video list in order of concatenation
video_list = ["video1.mov","black1.mov","video2.mov","black2.mov","video3.mov","black3.mov","video4.mov"]
#primary index is the video which serves as a "model" for reencoding. All other videos will have the same codec as this one. 
#Note the first video has a primary_index "1", not "0"
combine_video(video_list, primary_index=1, output_file="marines_5min_new.mov", crf=23, preset="fast")

In [ ]:
from video_commands import * 
video_list = ["video1.mov","black1.mov","video2.mov","black2.mov","video3.mov","black3.mov","video4.mov"]
#primary index is the video which serves as a "model" for reencoding. All other videos will have the same codec as this one. 
#Note the first video has a primary_index "1", not "0"
combine_video(video_list, primary_index=1, output_file="marines_5min_new.mov", crf=23, preset="fast")

## Separating different audio channels from a video
copy and paste the below to the terminal. Don't run it here! The output is in default folders <b><i>separated/htdemucs/marines_5min_new</i></b>

<i><b>demucs marines_5min_new.mov</i></b>

## extracting voice to subtitles

In [ ]:
#input can be a video or audio. If video, make sure the audio channel is "clean"
video = "separated/htdemucs/marines_5min_new/vocals.mp3" #the original file is vocals.wav. 
outputsrt = "subtitle.srt"
voice_to_srt(video, outputsrt)

In [ ]:
from video_commands import * 
video = ".mp3" #the original file is vocals.wav. 
outputsrt = ".srt"
voice_to_srt(video, outputsrt)

## Translation

<i>english = en<br>
simplified chinese = "zh-CN" for google, "zh" for deepl<br>
traditional chinese = "zh-TW" for google, "zh" for deepl<br>
japanese = ja<br>
korea = ko<br>
german = de<br>
french = fr<br></i>

In [ ]:
subtitle_file = "subtitle.srt" # the file we extract from the step above voice_to_srt()
outputfile = "subtitle_google_ja.srt"
target_lang = "ja"
source_lang = "en" #not inputting this would set the value to auto-detect
translate_srt_google(subtitle_file,outputfile, target_lang, source_lang)

In [ ]:
from video_commands import * 
subtitle_file = ".srt" # the file we extract from the step above voice_to_srt()
outputfile = "subtitle_google_.srt"
target_lang = ""
source_lang = "" #not inputting this would set the value to auto-detect
translate_srt_google(subtitle_file,outputfile, target_lang, source_lang)

In [ ]:
subtitle_file = "subtitle.srt"
outputfile = "subtitle_google_ja.srt"
target_lang = "ja" 
source_lang = "en" #not inputting this would set the value to auto-detect
deepl_translate_srt(subtitle_file, outputfile,target_lang)

In [ ]:
from video_commands import * 
subtitle_file = ".srt"
outputfile = "subtitle_deepl_.srt"
target_lang = "" 
source_lang = "" #not inputting this would set the value to auto-detect
deepl_translate_srt(subtitle_file, outputfile,target_lang)

## combining translations for proofreading

In order to make proofreading easier, you can combine the subtile in the original language and the translated language into one file. The file looks like this:<br><br>
1<br>
00:00:00.000 --> 00:00:03.399<br>
about to our rectum pivions instead of machine guns.<br>
マシンガンの代わりに直腸のピビオンを使う。<br><br>


In [ ]:
file1_path = "subtitle.srt" #input file, make this the original language
file2_path = "subtitle_deepl_ja.srt" #input file, make this the translated langauge
output_path = "subtitle_en_ja.srt" #output file which combines two input files
combine_srt(file1_path, file2_path, output_path)

In [ ]:
from video_commands import * 
file1_path = ".srt" #input file, make this the original language
file2_path = ".srt" #input file, make this the translated langauge
output_path = ".srt" #output file which combines two input files
combine_srt(file1_path, file2_path, output_path)

## Separating translation after proofreading

In [ ]:
input_file = "subtitle_en_ja.srt"
language_choice = 2
output_file = "subtitle_ja.srt"
separate_srt_languages(input_file, language_choice, output_file)

In [ ]:
from video_commands import * 
input_file = ".srt"
language_choice = 2
output_file = ".srt"
separate_srt_languages(input_file, language_choice, output_file)

## creating zoom video
create a rescaled image and numbered grid first


In [ ]:
main_video = "marines_5min_new.mov" #we need to rescale the input image to match the size of the main video
video_width,video_height = get_media_dimensions(main_video) 
image_name = "media_files/boat.jpg" #the input image
#the rescaled image which matches the dimensions of the main video. This file is used as the overlay
output_filename = "boat_rescaled.jpg" 
make_rescaled_image(video_width, video_height, image_name, output_filename) #produce the rescaled image
input_file = output_filename
output_file = "boat_rescaled_grid.jpg"
#add a grid to the rescaled image to retrieve pixel positions, setting an interval = 50 means the distance between the grid is 50 pixels
add_numbered_grid(input_file, output_file,video_width,video_height,interval=50,line_color="red",number_color="yellow") 


In [ ]:
from video_commands import * 
main_video = "" #we need to rescale the input image to match the size of the main video
video_width,video_height = get_media_dimensions(main_video) 
image_name = " #the input image
#the rescaled image which matches the dimensions of the main video. This file is used as the overlay
output_filename = "" 
make_rescaled_image(video_width, video_height, image_name, output_filename) #produce the rescaled image
input_file = output_filename
output_file = ""
#add a grid to the rescaled image to retrieve pixel positions, setting an interval = 50 means the distance between the grid is 50 pixels
add_numbered_grid(input_file, output_file,video_width,video_height,interval=50,line_color="red",number_color="yellow") 


<b>Top left corner has x = 0, y = 0 </b>. 

In [ ]:
x_pos,y_pos = 275,275
duration = 5
zoom_start = 2
zoom_max = 2
out_file = "boat_video.mov"
image_list = [["boat_rescaled.jpg", x_pos, y_pos, duration, zoom_start, zoom_max, out_file]]
create_zoom(image_list, main_video, crf=18, preset="fast")

In [ ]:
from video_commands import * 
x_pos,y_pos = ,
duration = 5
zoom_start = 2
zoom_max = 2
out_file = ".mov"
image_list = [[".jpg", x_pos, y_pos, duration, zoom_start, zoom_max, out_file]]
create_zoom(image_list, main_video, crf=18, preset="fast")

## Overlaying everything


para_list = <br>
<i>[["media_files/Aumun_Background.mov","0:20","0:30","0:05",0],</i>&nbsp;&nbsp;&nbsp;&nbsp;#0:20-0:30 of the main video, extract 0:05-0:15 of the overaly, no fade-out 
           <i>  ["boat_video.mov","3:30","3:35","0:00",0], </i>&nbsp;&nbsp;&nbsp;&nbsp;#3:30-3:35 of the main video, extract 0:00-0:05 of the overaly, no fade-out<br>
         <i>    ["media_files/Mountain_Forest.mov","4:00","4:15","0:00",3],</i>&nbsp;&nbsp;&nbsp;&nbsp; #4:00-4:15 of the main video, extract 0:00-0:15 of the overaly, fade-out = 3s<br>
          <i>    ["media_files/kochi.jpg","1:50","1:55","2"],</i>&nbsp;&nbsp;&nbsp;&nbsp;#1:50-1:55 of the main video, fade-out = 2s<br>
          <i>    ["media_files/bridge.jpg","2:00","2:15","5"],</i>&nbsp;&nbsp;&nbsp;&nbsp;#2:00-2:15 of the main video, fade-out = 5s<br>
          <i>    ["media_files/cat.jpg","3:00","3:03","0"],</i>&nbsp;&nbsp;&nbsp;&nbsp;#3:00-3:03 of the main video, no fade-out<br>
           <i>   ["media_files/Mozart.wav","0:10","3:00",5,5,3],</i>&nbsp;&nbsp;&nbsp;&nbsp;#0:10-3:00 of the main video, fade in = fade-out = 5s,3 times audio volume<br>
           <i>   ["media_files/Rachmaninoff.wav","3:10",f"{end_time}",0,0,1]]</i>&nbsp;&nbsp;&nbsp;&nbsp;#3:10-the end of the main video, no fadein fadeout,original audio volume<br>


</i>

Let's create the final product.<br>

In [ ]:
main_video = "marines_5min_new.mov"
end_time = get_video_length(main_video)
para_list = [["media_files/Aumun_Background.mov","0:20","0:30","0:05",0],
["boat_video.mov","3:30","3:35","0:00",0],
["media_files/Mountain_Forest.mov","4:00","4:15","0:00",3],
["media_files/kochi.jpg","1:50","1:55","2"],
["media_files/bridge.jpg","2:00","2:15","5"],
["media_files/cat.jpg","3:00","3:03","0"],
["media_files/Mozart.wav","0:10","3:00",5,5,3],
["media_files/Rachmaninoff.wav","3:10",f"{end_time}",0,0,1]]
output_file = "marines_5min_all_overlay.mov"
overlay_video_img_music(main_video, para_list, output_file, crf=23, preset="fast")

In [ ]:
from video_commands import * 
main_video = ""
end_time = get_video_length(main_video)
para_list = [[".mov","0:20","0:30","0:05",0],
[".mov","3:30","3:35","0:00",0],
[".mov","4:00","4:15","0:00",3],
[".jpg","1:50","1:55","2"],
[".jpg","2:00","2:15","5"],
[".jpg","3:00","3:03","0"],
[".mp3","0:10","3:00",5,5,3],
[".mp3","3:10",f"{end_time}",0,0,1]]
output_file = ".mov"
overlay_video_img_music(main_video, para_list, output_file, crf=23, preset="fast")

## extracting screen shot

In [ ]:
#extract screen shot at 1:32 and name it to mosaic1.png, another at 0:10 and name it to mosiac2.png
main_video = "marines_5min_all_overlay.mov"
time_and_name_list = [["1:32","mosaic1.png"],["0:10","mosaic2.png"]] #list of list
extract_frames(main_video, time_and_name_list, fast_seek=True)
input_file = ["mosaic1.png","mosaic2.png"]
output_file = ["mosaic1_grid.png","mosaic2_griod.png"]
video_width,video_height = get_media_dimensions(main_video)
for i,n in zip(input_file,output_file):
      video_width,video_height = get_media_dimensions(main_video)
      add_numbered_grid(i, n,video_width,video_height,interval=50,line_color="red",number_color="black")



In [ ]:
from video_commands import * 
#extract screen shot at 1:32 and name it to mosaic1.png, another at 0:10 and name it to mosiac2.png
main_video = ".mov"
time_and_name_list = [["1:00",".png"],["0:10",".png"]] #list of list
extract_frames(main_video, time_and_name_list, fast_seek=True)
input_file = ["mosaic1.png","mosaic2.png"]
output_file = ["mosaic1_grid.png","mosaic2_griod.png"]
video_width,video_height = get_media_dimensions(main_video)
for i,n in zip(input_file,output_file):
      video_width,video_height = get_media_dimensions(main_video)
      add_numbered_grid(i, n,video_width,video_height,interval=50,line_color="red",number_color="black")



 ## adding mosaic
 [25, 25, 100, 50, 15, 10, 15]<br>
 xpos,ypos,width,length,pixelation, start time, end time
 
 

In [ ]:
main_video = "marines_5min_all_overlay.mov"
mosaic_list=[
[25, 25, 100, 50, 15, 10, 15],
[130, 175, 270, 120, 15, "1:30", "1:35"]]
output_video = "marines_5min_all_overlay_mosaic.mov"
apply_mosaics(main_video, output_video, mosaic_list)

In [ ]:
from video_commands import * 
main_video = ".mov"
mosaic_list=[
[25, 25, 100, 50, 15, 10, 15],
[130, 175, 270, 120, 15, "1:30", "1:35"]]
output_video = ".mov"
apply_mosaics(main_video, output_video, mosaic_list)

## srt to ass


In [ ]:
srt_file = "subtitle.srt" #input file
ass_file = "subtitle.ass" #output file
main_video = "marines_5min_all_overlay_mosaic.mov"
#the ass file has a resolution which is the same as the dimension of the main video
video_width, video_height = get_media_dimensions(main_video)
srt_to_ass(srt_file, ass_file, video_width, video_height, fontname="Arial", fontsize=24)

In [ ]:
from video_commands import * 
srt_file = ".srt" #input file
ass_file = ".ass" #output file
main_video = ".mov"
#the ass file has a resolution which is the same as the dimension of the main video
video_width, video_height = get_media_dimensions(main_video)
srt_to_ass(srt_file, ass_file, video_width, video_height, fontname="Arial", fontsize=24)

## Burning in subtitle

In [ ]:
subtitle_file = "subtitle.ass"
main_video = "marines_5min_all_overlay_mosaic.mov"
output_video = "marines_subtitle.mov"
#the font here is only for srt files
burn_subtitles(main_video, subtitle_file, output_video, font="Arial", crf =23, preset="fast")

In [ ]:
from video_commands import * 
subtitle_file = ".ass"
main_video = ".mov"
output_video = ".mov"
#the font here is only for srt files
burn_subtitles(main_video, subtitle_file, output_video, font="Arial", crf =23, preset="fast")

## Creating End Credits 
<i>contributors.txt</i>:<br><br>
<i>
Director: XXX<br>
Production Crew: AAA<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;ABC<br>
Music: XYZ</i><br><br>



In [ ]:
template_file = "end_credits_template.ass" # a template used everytime, txt_file below would be inserted
txt_file = "contributors.txt" #insert the content of this to the template above
output_file = "end_credits.ass" # This includes the above two files
width, height = get_media_dimensions(main_video)
#create an ass file
end_credits_ass(template_file, txt_file, output_file, width, height,
                   first_start="0:00:01.40", each_duration_s=6.0,
                   line_offset_s=0.60,
                   fallback_duration_s=4.0)
#embed the ass file to the video
ass_file = output_file
song_file = "media_files/Mozart.wav"
output_file = "end_credits_video.mov"
create_ending_film(ass_file, song_file, output_file, width, height,
                       volume=1.0, end_second=5, fadein=5, fadeout=10,
                       bg_video=None,video_fadeout=4)

In [ ]:
from video_commands import * 
template_file = "end_credits_template.ass" # a template used everytime, txt_file below would be inserted
txt_file = "contributors.txt" #insert the content of this to the template above
output_file = "end_credits.ass" # This includes the above two files
width, height = get_media_dimensions(main_video)
#create an ass file
end_credits_ass(template_file, txt_file, output_file, width, height,
                   first_start="0:00:01.40", each_duration_s=6.0,
                   line_offset_s=0.60,
                   fallback_duration_s=4.0)
#embed the ass file to the video
ass_file = output_file
song_file = "music/end_song.mp3"
output_file = ".mov"
volume = 
create_ending_film(ass_file, song_file, output_file, width, height,
                       volume=volume, end_second=5, fadein=5, fadeout=5,
                       bg_video=None,video_fadeout=4)

## final concatenation


In [ ]:
main_video = "marines_subtitle.mov"
video_list = [main_video,"end_credits_video.mov"]
output_file = "marines_5min_final.mov"
combine_video(video_list, primary_index=1, output_file=output_file, crf=23, preset="fast")

In [ ]:
from video_commands import * 
main_video = ".mov"
video_list = ["beginning_ban.mp4",main_video,"end_credits_video.mov"]
output_file = "_final.mov"
primary_index = 
combine_video(video_list, primary_index=primary_index, output_file=output_file, crf=23, preset="fast")

## parse time

In [ ]:
from video_commands import * 
t = "2:54"
time = parse_time(t) #result in seconds

## video length
result in mm:ss

In [ ]:
from video_commands import * 
input_file = ""
get_video_length(input_file)

## media dimensions

In [ ]:
from video_commands import * 
file_path = ""
width, height = get_media_dimensions(file_path)
a_width, a_height = get_media_active_dimensions(file_path, threshold=16, sample_frames=5):
print("normal w h:",width,height)
print("active w h:",a_width,a_height)

## mp3 to wav

In [ ]:
from video_commands import * 
mp3_file = ""
wav_file = mp3_to_wav(mp3_file):

## adjust wav volume

In [ ]:
from video_commands_mine import * 
wav_file = ""
volume_factor = 
output = adjust_wav_volume(wav_file, volume_factor)

## extract pages to txt
command line

In [ ]:
chmod +x pages2txt.sh
./pages.txt xxx.pages

## pip3 download

In [ ]:
!pip3 install --break-system-packages plistlib